In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/HOGxSVM') # change working directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import numpy
import google.colab.patches as colab
from MNIST import MNIST

In [4]:
mnist_train = MNIST('/content/drive/MyDrive/Colab Notebooks/HOGxSVM/train-images.idx3-ubyte',
'/content/drive/MyDrive/Colab Notebooks/HOGxSVM/train-labels.idx1-ubyte')

In [5]:
image,label = mnist_train.getitem(10)

In [6]:
colab.cv2_imshow(image)
print(f'{label}:{image.shape}')

3:(28, 28, 1)


In [7]:
winSize = (28,28)
blockSize = (28,28)
blockStride = (1,1)
cellSize = (14,14)
nbins = 9
HOG = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
n_train = len(mnist_train)
train_image_mat = numpy.zeros((n_train, 36), dtype=numpy.float32)
train_label_mat = numpy.zeros((n_train),dtype=int)

In [9]:
for index in range(n_train):
  # get an item
  image, label = mnist_train.getitem(index)
  # apply HOG descriptor
  a_hog = HOG.compute(image)
  a_hog = a_hog.reshape(-1)
  a_hog = a_hog.astype(numpy.float32)
  # put the data to numpy array
  train_image_mat[index] = a_hog
  train_label_mat[index] = label

In [10]:
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 10000, 1e-6))

In [11]:
svm.train(train_image_mat, cv2.ml.ROW_SAMPLE, train_label_mat)

True

In [15]:
mnist_test = MNIST("/content/drive/MyDrive/Colab Notebooks/HOGxSVM/t10k-images.idx3-ubyte",
"/content/drive/MyDrive/Colab Notebooks/HOGxSVM/t10k-labels.idx1-ubyte")

In [16]:
n_test = len(mnist_test)
n_correct = 0
n_wrong = 0
for index in range(n_test):
  # get an item
  image, label = mnist_test.getitem(index)

  # apply HOG descriptor
  a_hog = HOG.compute(image)
  a_hog = a_hog.reshape((1,-1))
  a_hog = a_hog.astype(numpy.float32)
  result = svm.predict(a_hog)
  result = int(result[1])
  if result == label:
    n_correct = n_correct + 1
  else:
    n_wrong = n_wrong + 1

In [18]:
accuracy = n_correct / n_test
print(f'{n_correct}')
print(f'accuracy:{accuracy*100.}')

9261
accuracy:92.61
